# Grid-Based Path Planning: Potential Fields vs Wavefront Planner

This notebook demonstrates the core functionality of the grid-based path planning algorithms implemented in this project. It compares the classical Harmonic Potential Field approach against the Wavefront Planner regarding trajectory generation, local minima avoidance, and connectivity.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Add source directory to python path
sys.path.append(os.path.abspath('../src'))

from attractive_field import compute_attractive_field
from brushfire import compute_obstacle_distance
from repulsive_field import compute_repulsive_field
from gradient_descent import compute_total_potential, gradient_descent_path
from wavefront import compute_wavefront
from greedy_descent import greedy_descent_path

## 1. Map Loading and Initialization
We load binary occupancy grids representing the operational environment.

In [ ]:
def load_map(file_path):
    image = Image.open(file_path).convert('L')
    grid_map = np.array(image.getdata()).reshape((image.size[1], image.size[0])) / 255.0
    grid_map[grid_map > 0.5] = 1
    grid_map[grid_map <= 0.5] = 0
    grid_map = (grid_map * -1) + 1
    return grid_map

map_file = '../data/map0.png'
grid_map = load_map(map_file)

start_pos = [10, 10]
goal_pos = [40, 110]

plt.figure(figsize=(6, 6))
plt.matshow(grid_map, fignum=0, cmap='gray_r')
plt.plot(start_pos[1], start_pos[0], 'go', markersize=10, label='Start')
plt.plot(goal_pos[1], goal_pos[0], 'r*', markersize=10, label='Goal')
plt.title("Initial Map Configuration", pad=15)
plt.legend()
plt.show()

## 2. Potential Field Generation
Computing the attractive and repulsive potentials to formulate the total navigation field.

In [ ]:
# 1. Attractive Field
attr_map = compute_attractive_field(grid_map, goal_pos, scaling_factor=0.01, mode='euclidean')

# 2. Brushfire Distance to Obstacle
dist_map = compute_obstacle_distance(grid_map, connectivity=8)

# 3. Repulsive Field
rep_map = compute_repulsive_field(dist_map + 1, influence_distance=10, scaling_factor=100)

# 4. Total Potential map
total_map = compute_total_potential(attr_map, rep_map)
total_vis = np.clip(total_map, a_min=None, a_max=np.percentile(total_map, 95))

plt.figure(figsize=(6, 6))
plt.matshow(total_vis, fignum=0)
plt.plot(goal_pos[1], goal_pos[0], 'r*', markersize=10, label='Goal')
plt.colorbar()
plt.title("Total Potential Field", pad=15)
plt.show()

## 3. Path Extraction
Comparing standard gradient descent (Potential Field) and greedy global descent (Wavefront Planner).

In [ ]:
# Gradient Descent extraction
path_xs, path_ys = gradient_descent_path(total_map, start_pos, goal_pos, connectivity=8, max_iters=2000)

# Wavefront Planner extraction
wf_map, _ = compute_wavefront(grid_map, goal_pos, connectivity=8)
wf_xs, wf_ys = greedy_descent_path(wf_map, start_pos, goal_pos, connectivity=8)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.matshow(grid_map, fignum=0, cmap='gray_r')
plt.plot(start_pos[1], start_pos[0], 'go', markersize=8)
plt.plot(goal_pos[1], goal_pos[0], 'r*', markersize=10)
plt.plot(path_ys, path_xs, 'b-', linewidth=2)
plt.title('Potential Field Path')

plt.subplot(1, 2, 2)
plt.matshow(grid_map, fignum=0, cmap='gray_r')
plt.plot(start_pos[1], start_pos[0], 'go', markersize=8)
plt.plot(goal_pos[1], goal_pos[0], 'r*', markersize=10)
plt.plot(wf_ys, wf_xs, 'b-', linewidth=2)
plt.title('Wavefront Planner Path')

plt.show()

## Technical Analysis

### Local Minima Behavior
Harmonic potential fields rely strictly on following the summation of independent vector forces: an attractive component toward the destination and a short-range repulsive force pushing away from immediate geometries. If a robot enters a concave obstacle structure directly blocking the linear Euclidean path towards the target, it converges upon a location where $\nabla U_{att} + \nabla U_{rep} = 0$. In mathematical terms, the negative gradient completely drops off long before reaching the goal, permanently stalling simple greedy step algorithms within a false energy well. Generating paths accurately through such regions requires external heuristics (e.g., stochastic boundary following).

### Connectivity Comparison
Constraining evaluation to a strict 4-neighbor grid topography introduces severe limitations on directionality. Traversal from origin $(0, 0)$ to coordinate $(10, 10)$ enforces $O(2N)$ stepped movements forming a rigid jagged path. By introducing full 8-neighbor matrices, Euclidean vectors align tighter with naturally continuous curved trajectories, greatly shrinking the mathematical variance between true operational paths and their discretized estimations. 

### Why Wavefront Works
The wavefront topological transform actively bypasses localized saddle points by fundamentally restructuring the mapping constraint equation. Utilizing a globally expanded breadth-first metric seeded at $q_{goal}=0$, the grid universally develops strictly monotonic ascending values. Due to its exhaustive breadth-bound iteration extending around structural barriers, there exists no coordinate containing neighboring grid cells with singularly identical or rising uniform distances. Local minima physically cannot synthesize since a smaller integer value guiding down towards $0$ is perpetually guaranteed to exist adjacently.